In [ ]:
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import igraph as ig

from config import DATA_PATH, FIGURE_PATH
from tests.VAT import *
from visual_flow_diagram_helpers import *
from NW_validation_modules import *
from analyses.draw_geomap_subnetwork import canton_aggregated_count_and_column, load_shapefiles

## Feed in production network(s)

In [ ]:
years = range(2007,2015+1)

banana_pn_edgelists = pd.DataFrame()
banana_pn_nodelists = pd.DataFrame()
global_edgelists = {}
for y in years:
    banana_pn = ig.read(filename=DATA_PATH / f'pn{y}.graphml', format='graphml')
    banana_pn_edgelists = pd.concat([banana_pn_edgelists, banana_pn.get_edge_dataframe()])
    banana_pn_nodelists = pd.concat([banana_pn_nodelists, banana_pn.get_vertex_dataframe()])
    global_edgelists[y] = pd.read_csv(DATA_PATH / 'firm-level' / f'global_edgelist_{y}.csv', dtype={'id_supplier':str, 'id_customer':str})

In [ ]:
#existed_nodes_df = banana_pn_nodelists[['firm_id', 'ISIC4', 'descrip_n4', 'ADM2', 'ADM1']].drop_duplicates()

# Pivot countrywide companies
cw_nodes = load_firm_data_cw(contr_type='sociedades')
ID_TO_SECTOR = dict(zip(cw_nodes["firm_id"], cw_nodes["ISIC4"]))
SECTOR_TO_DESCR = dict(zip(cw_nodes["ISIC4"], cw_nodes["descrip_n4"]))
ID_TO_ADM = dict(zip(cw_nodes["firm_id"], zip(cw_nodes["ADM2"], cw_nodes["ADM1"])))

sector_color_map = {'A0122':"#ffe135", 'G4630':"#3bba1e",
                    'A0163':"#FFB922", 'G4620':"#3bba1e",
                    'C2220':"#2669AC", 'G4669':"#303030",
                    'G4772':"#158C92", 'S9609':"#AA29B3",
                    'C1702':"#9A7444", 'C1701':"#9A7444",
                    'H4923':"#A0A0A0",
                    }

## Validating

### Money flows diagram

In [ ]:
for y in years:
    if y != 2015: continue

    # Custom IDs that will not be aggregated in the visualization
    firm_PN_visual_IDs = set([
        '1797601', '1899534'
        ])

    validate_ids(firm_PN_visual_IDs, set(banana_pn_nodelists[banana_pn_nodelists['date']==y]['firm_id']))
    visible_nodes_df, visible_flows_df = derive_visualization_data(global_edgelists[y], firm_PN_visual_IDs,
                                                                   ID_TO_SECTOR, SECTOR_TO_DESCR, ID_TO_ADM, sector_color_map,
                                                                   show_roe=True
                                                                   )
    
    plot_flows_sankey(visible_nodes_df, visible_flows_df)

### Prod. netw. I/O table

In [ ]:
for y in years:
    print(y)
    if y in [2015,2014]:
        pivot = compute_io_table_sector_level(banana_pn_edgelists[banana_pn_edgelists['date']==y])
        if pivot.empty:
            continue
        plot_io_pivottable(pivot, logscale=True,
                        xlabel= 'Customer sector', ylabel= 'Supplier sector', title=f'I/O table {y}')

### Capturing Banana growing and wholesale firms and locations (compared with offic. data)

In [ ]:
# Define your province colors upfront (modify as needed)
PROVINCE_COLORS = {
    "Guayas": plt.cm.tab10.colors[0],
    "El Oro": plt.cm.tab10.colors[1],
    "Los Rios": plt.cm.tab10.colors[2],
    "Pichincha": plt.cm.tab10.colors[3],
    "Manabi": plt.cm.tab10.colors[4],
    "Others": 'grey'    # Add more provinces as needed
}

In [ ]:
notable_prov = set(PROVINCE_COLORS.keys())

for sector in ['A0122', 'G4630']:    
    # Group sector PN data into (year, province)
    province_df_sect = group_sectorfirms_per_province(banana_pn_nodelists, sector=sector,
                                                      col_sum='cw_s_out',  # COUNTRYWIDE
                                                      notable_prov=notable_prov) 
    # Pivot prod. net. companies
    province_df_sect = province_df_sect[province_df_sect['element'] == 'count']
    pivot_1 = province_df_sect.pivot_table(index="year", columns="province", values="value", aggfunc='sum').fillna(0)

    # Load official data
    cfn_empresas = load_cfn_empresas_per_provincia(sector)
    pivot_2 = cfn_empresas.pivot_table(index="year", columns="province", values="value", aggfunc='sum').fillna(0)

    plot_stacked_bar_comparison(pivot_1, pivot_2, sector, column_color=PROVINCE_COLORS)

In [ ]:
sector = None#'A0122'
province_df_sect = group_sectorfirms_per_province(cw_nodes, sector=sector, col_sum='cw_s_out',
                                                  notable_prov=PROVINCE_COLORS.keys())

province_df_sect = province_df_sect[province_df_sect['element'] == 'count']
pivot_1 = province_df_sect.pivot_table(index="year", columns="province", values="value", aggfunc='sum').fillna(0)

In [ ]:
plot_stacked_bar(pivot_1, column_color=PROVINCE_COLORS, title_descr=f'Countrywide{"" if sector is None else " " + sector}')

### Micro-small-medium-large enterprise

In [ ]:
A0122_nodelist = banana_pn_nodelists[banana_pn_nodelists['ISIC4'] == 'A0122'].copy()

# Group by year and create rankings
fig, ax = plt.subplots(figsize=(6, 4))

for y in years:
    A0122_nodelist_y = A0122_nodelist[A0122_nodelist['date']==int(y)]
    col = 'cw_s_out'
    A0122_nodelist_y = A0122_nodelist_y.sort_values('cw_s_out', ascending=False).reset_index(drop=True)
    A0122_nodelist_y['rank'] = range(1, len(A0122_nodelist_y) + 1)

    ax.plot(A0122_nodelist_y['rank'], A0122_nodelist_y[col], marker='o', label=str(y), alpha=0.7)
ax.set(xlabel='Rank', ylabel=f'{col} Value', title=f'{col} Values by Rank for Each Year',
       #xscale='log', 
       yscale='log')
ax.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Export


### Banana cultivated area

### Where are A0122's customers in G4630 located? How much do they buy?

In [ ]:
adm1_geodf, adm2_geodf = load_shapefiles()

for y in years:
    if y!= 2015: continue
    pn_edgelist_y = global_edgelists[y][(global_edgelists[y]['inPN_supplier'] == True) & (global_edgelists[y]['inPN_customer'] == True)]
    A0122_to_G4630_flows_y = pn_edgelist_y[(pn_edgelist_y['sector_supplier'] == 'A0122') & (pn_edgelist_y['sector_customer'] == 'G4630')]
    A0122customers_inG4630 = set(A0122_to_G4630_flows_y['id_customer'].unique())
    A0122customers_inG4630_df = banana_pn_nodelists[(banana_pn_nodelists['date'] == y) & 
                                                    (banana_pn_nodelists['firm_id'].isin(A0122customers_inG4630))]
    A0122customers_inG4630_df = (A0122customers_inG4630_df.merge(
        A0122_to_G4630_flows_y.groupby('id_customer')['weight'].sum(), left_on='firm_id', right_on='id_customer')
        .rename(columns={'weight':'ban_supply'})
        .sort_values(by='ban_supply', ascending=False))

In [ ]:
canton_aggregated_count_and_column(A0122customers_inG4630_df, adm2_geodf, adm1_geodf,
                                   logscale=True, year=y, col_fun1 = ('ban_supply', 'sum'))

### Out of the PN edges

In [ ]:
year = 2015
global_edgelist_y = global_edgelists[year]
pn_and_roe_edgelist_y = global_edgelist_y[(global_edgelist_y['inPN_supplier'] == True) ^ (global_edgelist_y['inPN_customer'] == True)]

A0122customers_inroe = (pn_and_roe_edgelist_y[pn_and_roe_edgelist_y['sector_supplier'] == 'A0122']
                        .groupby('sector_customer', as_index=False)
                        .agg({'weight':'sum', 'sector_customer':'first','id_customer': 'nunique'})
                        .sort_values(by='weight', ascending=False))
A0122customers_inroe['sector_customer_descr'] = A0122customers_inroe['sector_customer'].map(SECTOR_TO_DESCR)

A0122suppliers_inroe = (pn_and_roe_edgelist_y[pn_and_roe_edgelist_y['sector_customer'] == 'A0122']
                        .groupby('sector_supplier', as_index=False)
                        .agg({'weight':'sum', 'sector_supplier':'first','id_supplier': 'nunique'})
                        .sort_values(by='weight', ascending=False))
A0122suppliers_inroe['sector_supplier_descr'] = A0122suppliers_inroe['sector_supplier'].map(SECTOR_TO_DESCR)

display(A0122customers_inroe.head(10))
display(A0122suppliers_inroe.head(10))

In [ ]:
C2220customers_inroe = (pn_and_roe_edgelist_y[pn_and_roe_edgelist_y['sector_supplier'] == 'C2220']
                        .groupby('sector_customer', as_index=False)
                        .agg({'weight':'sum', 'sector_customer':'first','id_customer': 'nunique'})
                        .sort_values(by='weight', ascending=False))
C2220customers_inroe['sector_customer_descr'] = C2220customers_inroe['sector_customer'].map(SECTOR_TO_DESCR)

C2220suppliers_inroe = (pn_and_roe_edgelist_y[pn_and_roe_edgelist_y['sector_customer'] == 'C2220']
                        .groupby('sector_supplier', as_index=False)
                        .agg({'weight':'sum', 'sector_supplier':'first','id_supplier': 'nunique'})
                        .sort_values(by='weight', ascending=False))
C2220suppliers_inroe['sector_supplier_descr'] = C2220suppliers_inroe['sector_supplier'].map(SECTOR_TO_DESCR)

#display(C2220suppliers_inroe.head(10))
#display(C2220customers_inroe.head(10))

In [ ]:
G4630customers_inroe = (pn_and_roe_edgelist_y[pn_and_roe_edgelist_y['sector_supplier'] == 'G4630']
                        .groupby('sector_customer', as_index=False)
                        .agg({'weight':'sum', 'sector_customer':'first','id_customer': 'nunique'})
                        .sort_values(by='weight', ascending=False)
                        )
G4630customers_inroe['sector_customer_descr'] = G4630customers_inroe['sector_customer'].map(SECTOR_TO_DESCR)

G4630suppliers_inroe = (pn_and_roe_edgelist_y[pn_and_roe_edgelist_y['sector_customer'] == 'G4630']
                        .groupby('sector_supplier', as_index=False)
                        .agg({'weight':'sum', 'sector_supplier':'first','id_supplier': 'nunique'})
                        .sort_values(by='weight', ascending=False)
                        )
G4630suppliers_inroe['sector_supplier_descr'] = G4630suppliers_inroe['sector_supplier'].map(SECTOR_TO_DESCR)

G4630suppliers_inroe['weight'].sum()/1e9

#display(G4630suppliers_inroe.head(15))
display(G4630customers_inroe.head(15))

In [ ]:
G4669customers_inroe = (pn_and_roe_edgelist_y[pn_and_roe_edgelist_y['sector_supplier'] == 'G4669']
                        .groupby('sector_customer', as_index=False)
                        .agg({'weight':'sum', 'sector_customer':'first','id_customer': 'nunique'})
                        .sort_values(by='weight', ascending=False)
                        )
G4669customers_inroe['sector_customer_descr'] = G4669customers_inroe['sector_customer'].map(SECTOR_TO_DESCR)

G4669suppliers_inroe = (pn_and_roe_edgelist_y[pn_and_roe_edgelist_y['sector_customer'] == 'G4669']
                        .groupby('sector_supplier', as_index=False)
                        .agg({'weight':'sum', 'sector_supplier':'first','id_supplier': 'nunique'})
                        .sort_values(by='weight', ascending=False)
                        )
G4669suppliers_inroe['sector_supplier_descr'] = G4669suppliers_inroe['sector_supplier'].map(SECTOR_TO_DESCR)

G4669suppliers_inroe['weight'].sum()/1e9

#display(G4669suppliers_inroe.head(15))
display(G4669customers_inroe.head(15))

In [ ]:
G4772suppliers_inroe = (pn_and_roe_edgelist_y[pn_and_roe_edgelist_y['sector_customer'] == 'G4772']
                        .groupby('sector_supplier', as_index=False)
                        .agg({'weight':'sum', 'sector_supplier':'first','id_supplier': 'nunique'})
                        .sort_values(by='weight', ascending=False)
                        )
G4772suppliers_inroe['sector_supplier_descr'] = G4772suppliers_inroe['sector_supplier'].map(SECTOR_TO_DESCR)
G4772suppliers_inroe